In [1]:

import tensorflow as tf
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
import numpy as np
# Set batch size for training and validation
batch_size = 32


In [3]:
gpus


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}


In [5]:

model_dictionary


{'DenseNet121': <function tensorflow.python.keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'DenseNet169': <function tensorflow.python.keras.applications.densenet.DenseNet169(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'DenseNet201': <function tensorflow.python.keras.applications.densenet.DenseNet201(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'EfficientNetB0': <function tensorflow.python.keras.applications.efficientnet.EfficientNetB0(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax', **kwargs)>,
 'EfficientNetB1': <function tensorflow.python.keras.applications.efficientnet.EfficientNetB1(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=

In [6]:
img_height = 224
img_width = 224
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "./archive/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Train/",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "./archive/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



Found 520 files belonging to 2 classes.
Using 104 files for validation.
Found 130 files belonging to 2 classes.
Using 104 files for training.


In [ ]:
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}
for model_name, model in tqdm(model_dictionary.items()):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)
    if 'NASNetLarge' in model_name:
        input_shape=(331,331,3)
        train_processed = train_ds
        validation_processed = val_ds
        print("nasa")
    else:
        input_shape=(224,224,3)
        train_processed = train_ds
        validation_processed = val_ds
        print("others")
    base_model = model(include_top=False,input_shape=input_shape)
    base_model.trainable = False
    print("models loaded")
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(len(train_ds.class_names))
    model = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(input_shape[0], input_shape[1],interpolation='bilinear', name=None),
        tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
        base_model,
        global_average_layer,
        tf.keras.layers.Dense(10),
        # tf.keras.layers.Dense(10),
        prediction_layer
    ])
    # model.summary()
    base_learning_rate = 0.001
    model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
    # We can evaluate the model right now to see how it does before training it on our new images
    initial_epochs = 1
    # validation_steps=20

    # loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)
    history = model.fit( train_ds ,
    epochs=initial_epochs,
    shuffle=True,
    batch_size=32,
    validation_data=val_ds,
    )
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(base_model.count_params())
    model_benchmarks['validation_accuracy'].append(history.history)
#     model_benchmarks['accuracy'].append(history.history['accuracy'][-1])


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

others
models loaded
4/4 [==============================] - 68s 9s/step - loss: 0.5962 - accuracy: 0.6598 - val_loss: 0.7486 - val_accuracy: 0.7885


  4%|██▉                                                                                | 1/28 [01:10<31:52, 70.82s/it]

others
models loaded
4/4 [==============================] - 42s 7s/step - loss: 0.6303 - accuracy: 0.7158 - val_loss: 0.6375 - val_accuracy: 0.6538


  7%|█████▉                                                                             | 2/28 [02:06<26:55, 62.13s/it]

others
models loaded
3/4 [=====================>........] - ETA: 0s - loss: 1.7450 - accuracy: 0.3750

In [ ]:

len(train_ds.class_names)


In [ ]:
# # custom modifications on top of pre-trained model
# clf_model = tf.keras.models.Sequential()
# clf_model.add(pre_trained_model)
# clf_model.add(tf.keras.layers.Dense(len(train_ds.class_names), activation='softmax'))
# clf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
# history = clf_model.fit(train_processed, epochs=3, validation_data=validation_processed,steps_per_epoch = 15)

#    # Calculate all relevant metrics
# model_benchmarks['model_name'].append(model_name)
# model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
# model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])


In [ ]:
model_benchmarks

In [ ]:

def fun(e,r):
    print(e)
    print(r)
    return(e)

In [ ]:
asd=0
IMG_SHAPE=(224,224,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights='imagenet')
base_model.summary()

In [ ]:
asd= train_ds.map(fun)

In [ ]:
a=cv2.imread('./archive/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Train/Glaucoma_Positive/036.jpg')

In [ ]:
feature_exteracted = base_model(np.array([cv2.resize(a,(224,224))]))

In [ ]:
np.array(feature_exteracted).shape


In [ ]:
1*7*7*1280

In [1]:
(224*224*3)/(1*7*7*1280)

2.4

In [ ]:
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}
for model_name, model in tqdm(model_dictionary.items()):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)
    if 'NASNetLarge' in model_name:
        input_shape=(331,331,3)
        train_processed = train_ds
        validation_processed = val_ds
        print("nasa")
    else:
        input_shape=(224,224,3)
        train_processed = train_ds
        validation_processed = val_ds
        print("others")
    base_model = model(include_top=False,input_shape=input_shape)
    base_model.trainable = False
    print("models loaded")
